# <font color='blue'> The Loop Converter Package (loop.py) </font>
<p>
<center>
<img width=700 src="https://humanities-web.s3.us-east-2.amazonaws.com/college/chicagostudies-prod/s3fs-public/2020-08/B%26W%20Chicago%20Map.jpg"></img>

In [97]:
from geopy.geocoders import Nominatim

def get_coordinates(address):
    geolocator = Nominatim(user_agent="address_converter")
    location = geolocator.geocode(address)
    
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None

def get_address_from_coordinates(latitude, longitude):
    geolocator = Nominatim(user_agent="address_converter")
    location = geolocator.reverse(f"{latitude}, {longitude}")

    address = location.raw['address']

    amenity = address.get('amenity', '')
    house_number = address.get('house_number', '')
    road = address.get('road', '')
    city = address.get('city', '')
    postcode = address.get('postcode', '')
    country = address.get('country', '')

    formatted_address = ''
    if amenity:
        formatted_address += f"{amenity}, "
    if house_number:
        formatted_address += house_number
        if road:
            formatted_address += ' ' + road
    elif road:
        formatted_address += road
    
    if city:
        formatted_address += f", {city}"
    if postcode:
        formatted_address += f", {postcode}"
    if country:
        formatted_address += f", {country}"

    precise_location = f"[{latitude}, {longitude}]"

    if not house_number:
        print("The precise location can also be found as:", precise_location)
    
    return formatted_address.strip(", ")

    

In [15]:
def calculate_blocks_away(latitude, longitude):
    # Miles per degree
    mpd = (24901.92 / 360)

    # Blocks per degree
    bpd_n_mr = mpd * 12
    bpd_n_rc = mpd * 10
    bpd_n_ct = mpd * 9
    bpd_n = mpd * 8
    bpd_w = mpd * 8

    roosevelt = 41.8674224
    cermak = 41.8528516
    thirtyfirst = 41.8382902
    
    # Baseline
    b_latitude, b_longitude = 41.88205727768228, -87.62783047240069


    if latitude > b_latitude:
        blocks_away_latitude = (latitude - b_latitude) * bpd_n
        direction_latitude = "north"
    elif roosevelt <= latitude <= b_latitude:
        blocks_away_latitude = (b_latitude - latitude) * bpd_n_mr
        direction_latitude = "south"
    elif cermak <= latitude <= roosevelt:
        blocks_away_latitude = ((b_latitude - roosevelt) * bpd_n_mr + (roosevelt - latitude) * bpd_n_rc)
        direction_latitude = "south"
    elif thirtyfirst <= latitude <= cermak:
        blocks_away_latitude = ((b_latitude - roosevelt) * bpd_n_mr + (roosevelt - cermak) * bpd_n_rc + (cermak - latitude) * bpd_n_ct)
        direction_latitude = "south"
    elif latitude <= thirtyfirst:
        blocks_away_latitude = ((b_latitude - roosevelt) * bpd_n_mr + (roosevelt - cermak) * bpd_n_rc + (cermak - thirtyfirst) * bpd_n_ct + (thirtyfirst - latitude) * bpd_n)
        direction_latitude = "south"

    if longitude >= b_longitude:
        blocks_away_longitude = abs(b_longitude - longitude) * bpd_w
        direction_longitude = "east"
    elif longitude <= b_longitude:
        blocks_away_longitude = abs(longitude - b_longitude) * bpd_w
        direction_longitude = "west"

    return blocks_away_latitude, direction_latitude, blocks_away_longitude, direction_longitude


In [75]:
# Miles per degree
mpd = (24901.92 / 360)

# Blocks per degree
bpd_n_mr = mpd * 12
bpd_n_rc = mpd * 10
bpd_n_ct = mpd * 9
bpd_n = mpd * 8
bpd_w = mpd * 6

roosevelt = 41.8674224
cermak = 41.8528516
thirtyfirst = 41.8382902

b_latitude, b_longitude = 41.88205727768228, -87.62783047240069
def calculate_latitude_longitude(blocks_away_latitude, direction_latitude, blocks_away_longitude, direction_longitude):
    latitude = longitude = None

    if direction_latitude == "north":
        latitude = b_latitude + (blocks_away_latitude / bpd_n)
    elif direction_latitude == "south":
        if blocks_away_latitude <= (b_latitude - roosevelt) * bpd_n_mr:
            latitude = b_latitude - (blocks_away_latitude / bpd_n_mr)
        elif blocks_away_latitude <= (b_latitude - roosevelt) * bpd_n_mr + (roosevelt - cermak) * bpd_n_rc:
            latitude = roosevelt - (blocks_away_latitude - (b_latitude - roosevelt) * bpd_n_mr) / bpd_n_rc
        elif blocks_away_latitude <= (b_latitude - roosevelt) * bpd_n_mr + (roosevelt - cermak) * bpd_n_rc + (cermak - thirtyfirst) * bpd_n_ct:
            latitude = cermak - (blocks_away_latitude - (b_latitude - roosevelt) * bpd_n_mr - (roosevelt - cermak) * bpd_n_rc) / bpd_n_ct
        else:
            latitude = thirtyfirst - (blocks_away_latitude - (b_latitude - roosevelt) * bpd_n_mr - (roosevelt - cermak) * bpd_n_rc - (cermak - thirtyfirst) * bpd_n) / bpd_n

    if direction_longitude == "east":
        longitude = b_longitude + (blocks_away_longitude / bpd_w)
    elif direction_longitude == "west":
        longitude = b_longitude - (blocks_away_longitude / bpd_w)

    return latitude, longitude

In [101]:
a = calculate_latitude_longitude(0, "north", 1000, "west")
lat = a[0]
long = a[1]
print(lat, long)
get_address_from_coordinates(lat, long)

41.88205727768228 -90.03728323748868


'10978 Union Grove Road, 61270, United States'

In [99]:
a = calculate_latitude_longitude(5, "north", 12, "west")
lat = a[0]
long = a[1]
get_address_from_coordinates(lat, long)

'1154 West Grand Avenue, Chicago, 60661, United States'

In [40]:
# Example usage
latitude, longitude = 41.7675, -87.679722

address = get_address_from_coordinates(latitude, longitude)
print(address)

# Example usage
latitude, longitude = 52.5211874,13.3949995

address = get_address_from_coordinates(latitude, longitude)
print(address)



# Example usage
address = "5500 South State Street, Chicago, IL"
coordinates = get_coordinates(address)

if coordinates:
    latitude, longitude = coordinates
    print(f"Latitude: {latitude}")
    print(f"Longitude: {longitude}")
else:
    print("Invalid address or could not find coordinates.")

# Example usage
b_latitude = 41.88205727768228  # Example value for b_latitude
b_longitude = -87.62783047240069  # Example value for b_longitude
latitude = 41.870000  # Example value for latitude
longitude = -87.640000  # Example value for longitude

blocks_away_lat, direction_lat, blocks_away_lon, direction_lon = calculate_blocks_away(latitude, longitude)
    
print(f"{blocks_away_lat} blocks {direction_lat}")
print(f"{blocks_away_lon} blocks {direction_lon}")


6928 South Bell Avenue, Chicago, 60636, United States
Hinter dem Zeughaus, Berlin, 10117, Deutschland
Latitude: 41.7941402
Longitude: -87.6259778
10.008312142065023 blocks south
6.734324504796817 blocks west
